In [50]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [51]:
from aicentro.session import Session
session = Session(verify=False)

In [62]:
import tensorflow as tf
import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.optimizers import RMSprop

In [63]:
voc_df = pd.read_csv('/aihub/workspace/AIProject_202007/Data/voc_preScale_final_2_20200721.csv', encoding = 'UTF-8')

In [64]:
del voc_df['Unnamed: 0']

In [65]:
voc_df

,상담대분류,상담중분류,상담소분류,상담일자,상품구분,급지,해지일,AS신청유형,AS원인유형,AS처리유형,AS처리시간,cnslt_rscs_diff,rscs_yn
0,4,10,27,2016-01-01,0,3,2016-06-15,25,86,68,-1.0,166,False
1,4,10,27,2016-01-01,0,4,2015-12-31,25,86,68,-1.0,-1,False
2,4,10,27,2016-01-01,0,2,1999-01-01,25,86,68,-1.0,-6209,False
3,2,11,56,2016-01-01,1,1,1999-01-01,18,75,55,16.0,-6209,False
4,4,10,27,2016-01-02,0,3,1999-01-01,25,86,68,-1.0,-6210,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,2,18,53,2020-07-14,1,4,1999-01-01,25,86,68,-1.0,-7865,False
40484,2,18,40,2020-07-14,1,2,1999-01-01,25,86,68,-1.0,-7865,False
40485,3,6,47,2020-07-14,1,3,1999-01-01,25,86,68,-1.0,-7865,False
40486,2,18,54,2020-07-14,1,6,1999-01-01,25,86,68,-1.0,-7865,False


In [66]:
voc_df.corr()

,상담대분류,상담중분류,상담소분류,상품구분,급지,AS신청유형,AS원인유형,AS처리유형,AS처리시간,cnslt_rscs_diff,rscs_yn
상담대분류,1.000000,-0.613951,-0.064542,-0.858035,-0.049287,0.100360,0.140734,0.132088,-0.050805,0.050805,-0.103012
상담중분류,-0.613951,1.000000,0.104928,0.309086,-0.018732,0.046618,0.075836,0.074865,-0.033488,-0.025873,0.123457
상담소분류,-0.064542,0.104928,1.000000,0.150294,-0.015071,0.088229,0.113209,0.117037,-0.069336,0.087981,0.096098
상품구분,-0.858035,0.309086,0.150294,1.000000,0.086309,-0.118083,-0.162363,-0.155013,0.053537,-0.025106,0.078850
급지,-0.049287,-0.018732,-0.015071,0.086309,1.000000,0.023191,0.014625,0.009234,0.040602,-0.048225,-0.046930
AS신청유형,0.100360,0.046618,0.088229,-0.118083,0.023191,1.000000,0.269823,0.255534,-0.082356,0.050427,0.017585
AS원인유형,0.140734,0.075836,0.113209,-0.162363,0.014625,0.269823,1.000000,0.911809,-0.137373,0.055328,0.015896
AS처리유형,0.132088,0.074865,0.117037,-0.155013,0.009234,0.255534,0.911809,1.000000,-0.126009,0.058493,0.019658
AS처리시간,-0.050805,-0.033488,-0.069336,0.053537,0.040602,-0.082356,-0.137373,-0.126009,1.000000,-0.022638,-0.016578
cnslt_rscs_diff,0.050805,-0.025873,0.087981,-0.025106,-0.048225,0.050427,0.055328,0.058493,-0.022638,1.000000,0.433840


In [67]:
val_columns = ['상담대분류', '상담중분류', '상담소분류', '급지', '상품구분', 'AS신청유형', 'AS원인유형', 'AS처리유형', 'rscs_yn']
voc_df = pd.DataFrame(voc_df, columns=val_columns)

In [68]:
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,4,10,27,3,0,25,86,68,False
1,4,10,27,4,0,25,86,68,False
2,4,10,27,2,0,25,86,68,False
3,2,11,56,1,1,18,75,55,False
4,4,10,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,2,18,53,4,1,25,86,68,False
40484,2,18,40,2,1,25,86,68,False
40485,3,6,47,3,1,25,86,68,False
40486,2,18,54,6,1,25,86,68,False


In [69]:
cnslt_big = [
'(인)고객불만'
,'가입해지'
,'고객불만'
,'고객불만 이관'
,'서비스 문의 및 신청'
,'품질'
,'회사 관련 문의'
]
toCharCoder_cnslt_big={i:k for i,k in enumerate(cnslt_big)}


In [70]:
l=[toCharCoder_cnslt_big[i] for i in voc_df['상담대분류']]
s=pd.Series(l)
voc_df['상담대분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,10,27,3,0,25,86,68,False
1,서비스 문의 및 신청,10,27,4,0,25,86,68,False
2,서비스 문의 및 신청,10,27,2,0,25,86,68,False
3,고객불만,11,56,1,1,18,75,55,False
4,서비스 문의 및 신청,10,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,18,53,4,1,25,86,68,False
40484,고객불만,18,40,2,1,25,86,68,False
40485,고객불만 이관,6,47,3,1,25,86,68,False
40486,고객불만,18,54,6,1,25,86,68,False


In [71]:
cnslt_mid = [
'AS처리점 유발'
,'CRG호전환'
,'고객센터/유통망관련문의'
,'과기정통부'
,'기타 단체'
,'내방고객'
,'대한주부클럽연합회'
,'방송통신위원회'
,'본사 전화민원'
,'서비스 문의 및 신청'
,'서비스 신청'
,'설치(이전)/AS'
,'소비자보호원'
,'소비자신문'
,'소비자연맹'
,'수신기'
,'수신장애'
,'시민의모임'
,'영업'
]

toCharCoder_cnslt_mid={i:k for i,k in enumerate(cnslt_mid)}

In [72]:
l=[toCharCoder_cnslt_mid[i] for i in voc_df['상담중분류']]
s=pd.Series(l)
voc_df['상담중분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,27,3,0,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,27,4,0,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,27,2,0,25,86,68,False
3,고객불만,설치(이전)/AS,56,1,1,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,53,4,1,25,86,68,False
40484,고객불만,영업,40,2,1,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,47,3,1,25,86,68,False
40486,고객불만,영업,54,6,1,25,86,68,False


In [73]:
cnslt_small = [
'(AS기사) 방문약속 미이행'
,'(AS기사) 방문약속 미이행'
,'(AS기사) 불친절'
,'(AS기사) 처리지연'
,'(AS기사) 후처리 미흡'
,'(문의) 시청불가'
,'(문의) 신호미약'
,'(설치기사) 방문약속 미이행'
,'(설치기사) 불친절'
,'(설치기사) 처리지연'
,'(설치기사) 후처리 미흡'
,'(접수) 기타메시지'
,'(접수) 리모콘 동작불가'
,'(접수) 수신기 동작불가'
,'(접수) 수신설비 파손 분실'
,'(접수) 신호미약'
,'(접수) 영상불량'
,'(접수) 음성불량'
,'(접수)스마트카드 오류 메시지'
,'AS 중복발생'
,'AS비용불만(즉납)'
,'AS점 직원 불친절'
,'AS점불만'
,'AS지연'
,'AS출동지연(도서지역)'
,'AS출동지연(약속미이행)'
,'AS출동지연(중복)SMS'
,'SkyChoice'
,'가격정책(제도)'
,'가전결합 불만'
,'개인정보 침해'
,'고객재산 파손 및 손괴'
,'기사칭찬'
,'기업 상품'
,'기타 단체'
,'기타(특이사례)'
,'명의도용'
,'방송품질'
,'사은품 미지금(약속미준수)'
,'상담'
,'상품/약정안내 미흡'
,'설치(이전)/AS'
,'설치점 불만'
,'설치점불만'
,'설치지연'
,'쌍방향 서비스'
,'업셀링 요금/약정 안내 미흡'
,'영업'
,'영업점불만'
,'영업점불만(신청서 대필)'
,'영업점불만(오처리/처리누락)'
,'영업점불만(임의개통_가개통)'
,'영업점불만(임의개통_명의도용)'
,'영업점불만(임의개통_이중가입)'
,'영업점불만(임의개통_일방적가입)'
,'이용불편'
,'잦은 AS'
,'청구/환불'
,'콘텐츠'
,'클레임처리지연'
,'패키지 체계 및 요금'
,'패키지 추가'
,'해지'
,'해지 접수'
]

toCharCoder_cnslt_small={i:k for i,k in enumerate(cnslt_small)}

In [74]:
l=[toCharCoder_cnslt_small[i] for i in voc_df['상담소분류']]
s=pd.Series(l)
voc_df['상담소분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,0,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,0,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,0,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,1,1,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,1,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,2,1,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,1,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,1,25,86,68,False


In [75]:
prdt_type = [
'없음'
,'위성'
,'인터넷'
]

toCharCoder_prdt_type={i:k for i,k in enumerate(prdt_type)}

In [76]:
l=[toCharCoder_prdt_type[i] for i in voc_df['상품구분']]
s=pd.Series(l)
voc_df['상품구분'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,없음,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,1,위성,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,위성,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,2,위성,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,위성,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,위성,25,86,68,False


In [77]:
region_lvl = [
'1'
,'2'
,'3'
,'4'
,'5'
,'6'
,'7'
,'8'
,'0'
]

toCharCoder_region_lvl={i:k for i,k in enumerate(region_lvl)}

In [78]:
l=[toCharCoder_region_lvl[i] for i in voc_df['급지']]
s=pd.Series(l)
voc_df['급지'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5,없음,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,2,위성,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5,위성,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,3,위성,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,4,위성,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7,위성,25,86,68,False


In [79]:
as_apply = [
'B/S점검'
,'IP백업고객 B/S점검'
,'IP연결요청'
,'PPV 오류 메시지'
,'PVR 녹화/시청 불가'
,'TQC'
,'USB 플레이어/타임머신 동작 불가'
,'VOD 시청 불가'
,'Wifi 사용불가'
,'Wifi 연결안됨'
,'[휴일]수신기 전원 미작동'
,'기타'
,'기타'
,'기타 메시지'
,'네트워크 연결확인 메시지'
,'리모콘 동작불가'
,'리모콘 사용법 모름'
,'모뎀 전원 미동작'
,'수신기 동작불가'
,'수신기 업그레이드안됨'
,'수신기 전원 미동작'
,'수신설비 파손 분실'
,'스마트카드 오류 메시지'
,'신호미약 메시지'
,'실버리모콘'
,'없음'
,'영상 불량'
,'영상 안아놈'
,'음성 불량'
,'음성 안나옴'
,'인터넷 사용불가'
,'재해지역AS발생'
,'케이블 연결방법 모름'
]

toCharCoder_as_apply={i:k for i,k in enumerate(as_apply)}

In [80]:
l=[toCharCoder_as_apply[i] for i in voc_df['AS신청유형']]
s=pd.Series(l)
voc_df['AS신청유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5,없음,없음,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,2,위성,수신기 동작불가,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5,위성,없음,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,3,위성,없음,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,4,위성,없음,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7,위성,없음,86,68,False


In [81]:
as_cause = [
'A/S취소'
,'A/V케이블 분리/단선'
,'A/V케이블 불량'
,'A/V케이블 오연결'
,'AS관련 클레임'
,'B/S점검'
,'IP연결요청'
,'LNB 동작불가'
,'LNB 파손'
,'LNB 파손/분실.동작불가'
,'LNB불량'
,'LTE모뎀 동작불가'
,'TP유니트 불량'
,'UTP 케이블 분리/단선'
,'UTP커넥터불량'
,'[FTTH] 포트불량'
,'[FTTH] 환경설정오류'
,'[IP공유기] 고객부주의(사용미숙)'
,'[IP공유기] 접속불량'
,'[IP공유기] 제품불량(제조사)'
,'[NE] 네트워크장비장애'
,'[Ntopia] 불복구'
,'[Ntopia] 포트불량'
,'[TOK] 비고장'
,'[고객취소] 비고장'
,'[구내단자함] 접속불량'
,'[기술상담] 문의성'
,'[기타] 문의성'
,'[기타] 비고장'
,'[모뎀] 고객부주의(사용미숙)'
,'[모뎀] 분실'
,'[모뎀] 불복구'
,'[모뎀] 속도저하'
,'[모뎀] 장애/공사/피해'
,'[모뎀] 접속불량'
,'[모뎀] 제품불량(제조사)'
,'[모뎀] 환경설정오류'
,'[세대단자함] 공법불량'
,'[세대단자함] 접속불량'
,'[실내선] 고객부주의(사용미숙)'
,'[실내선] 장애/공사/피해'
,'[실내선] 접속불량'
,'[인입광] 고객요청(민원)'
,'[인입광] 공법불량'
,'[인입광] 시설노후'
,'[인입광] 장애/공사/피해'
,'[인입광] 접속불량'
,'[인입광] 고객부주의(사용미숙)'
,'[인입선] 시설노후'
,'[인입선] 절연불량'
,'[인입선] 접속불량'
,'[자연회복] 자연회복'
,'[콘넥터] 접속불량'
,'[폐문] 비고장'
,'가스배관 등 위험지역 설치'
,'공유기 사용 미숙지'
,'공유기/허브 동작 불가'
,'기타'
,'녹화방법 미숙지'
,'도시가스배관정비'
,'동작 불가'
,'동작불가'
,'동툭케이블 분리/단선'
,'동축콘넥터 접불'
,'리모콘 동작불가'
,'리모콘 분실'
,'리모콘 추가'
,'리모콘 파손'
,'리모콘사용법 미숙지'
,'방수처리 불량'
,'불량'
,'설치관련 클레임'
,'수신기 LNB전원공급 불가'
,'수신기 동작불가'
,'수신기 분실'
,'수신기 전원 동작불가'
,'수신기 파손'
,'수신기 파손/분식/동작불가'
,'스마트카드 동작불가'
,'스마트카드 분실'
,'신호 불량'
,'안테나 고장불량(앙카볼트 미사용 등)'
,'안테나 동갖불가'
,'안테나 방향 틀어짐'
,'안테나 방향 틀어짐(고객사유)'
,'안테나 파손/분실'
,'없음'
,'일시적 장애'
,'잭 접불'
,'저잡음증폭기 불량'
,'전산 작업 오류'
,'전원'
,'회선 분리/단선'
,'접속불량'
,'정상확인'
,'지상파HD채널설정방법 미숙지'
,'자상파전환방법 미숙지'
,'취약지역 안테나 설치'
,'케이블연결방법 미숙지'
,'타 가입자 안테나 분배 설치'
,'파손/분실/동작불가'
,'해상도 변경 미숙지'
]

toCharCoder_as_cause={i:k for i,k in enumerate(as_cause)}

In [82]:
l=[toCharCoder_as_cause[i] for i in voc_df['AS원인유형']]
s=pd.Series(l)
voc_df['AS원인유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,없음,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5,없음,없음,없음,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,없음,68,False
3,고객불만,설치(이전)/AS,잦은 AS,2,위성,수신기 동작불가,수신기 전원 동작불가,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,없음,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5,위성,없음,없음,68,False
40484,고객불만,영업,상품/약정안내 미흡,3,위성,없음,없음,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,4,위성,없음,없음,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7,위성,없음,없음,68,False


In [83]:
as_process = [
'(외부자산)시설교체 및 수리 안내'
,'A/S취소'
,'A/V케이블 교체'
,'A/V케이블 재연결'
,'AS관련 클레임 처리'
,'AS취소'
,'IP연결요청'
,'LNB교체'
,'LNB교체'
,'LTE모뎀 교체'
,'Modulator 교체'
,'UTP 케이블 포설'
,'UTP커넥터 접속'
,'[FTTH] 포트교체'
,'[FTTH] 프로파일재설정'
,'[IP공유기] 교체(모뎀/ONT/STB)'
,'[IP공유기] 수리의뢰(고객)'
,'[IP공유기] 환경재설정'
,'[NE] 무출동네트워크장비복구'
,'[Ntopia] 포트교체'
,'[Ntopia] 프로파일재설정'
,'[구내단자함] 재접속'
,'[모뎀] 교체(모뎀/ONT/STB)'
,'[모뎀] 불복구제거(RESET)'
,'[모뎀] 재접속'
,'[모뎀] 환경재설정'
,'[세대단자함] 선로재구성(인입선 등)'
,'[세대단자함] 재접속'
,'[실내선]LAB케이블 교체'
,'[실내선]선로재구성(인입선 등)'
,'[실내선]재접속'
,'[인입광] 광콘넥터재접속'
,'[인입광] 교체(모뎀/ONT/STB)'
,'[인입광] 선로재구성(인입성 등)'
,'[인입광] 회선정비'
,'[인입선] 선로재구성(인입선 등)'
,'[인입선] 재접속'
,'[콘넥터] 광콘넥터재접속'
,'[콘넥터] 재접속'
,'공유기 사용안내'
,'공유기/허브 전원 재연결'
,'교체'
,'교체-IF증폭기-기타(미승인)'
,'교체-IF증폭기-아진'
,'교체-IF증폭기-청화'
,'교체/수리 안내'
,'기 타'
,'기술상담'
,'기타'
,'녹화방법안내'
,'동축케이블 포설'
,'동축콘넥터 접속'
,'리모콘 교체'
,'리모콘 추가'
,'리모콘사용법 안내'
,'무상수리교체(무료보증기간)'
,'방수(실리콘)처리'
,'배선 등 현장점검'
,'설치관련 클레임 처리'
,'수신기 교체'
,'수신기 전원 재연결'
,'스마트카드 교체'
,'안테나 교체'
,'안테나 교체/설치'
,'안테나 방향 조정'
,'안테나 재설치 또는 점검'
,'안테나재설치'
,'잉카볼트사용 및 고정상태'
,'없음'
,'유니트 교체'
,'유상구매교체(수리불가)'
,'유상수리교체(수리후재설치)'
,'자가보유교체(고객보유분)'
,'자동복구'
,'자연회복'
,'저잡음증폭기교체'
,'전산 작업'
,'전산작업'
,'전원 재입전'
,'전화선 연결'
,'정상확인/종료'
,'조치없음'
,'증폭기 신호조정'
,'증폭기전원재연결'
,'증폭기 추가(지사통보)'
,'지상파HD채널설정방법 안내'
,'지상파 전환방법 안내'
,'커넥터 재작업(재접속)'
,'케이블 연결방법 안내'
,'케이블 재포설(교체)'
,'타센터 AS처리'
,'해상도 변경 안내'
,'현장수리완료(OTA전송)']

toCharCoder_as_process={i:k for i,k in enumerate(as_process)}

In [84]:
l=[toCharCoder_as_cause[i] for i in voc_df['AS처리유형']]
s=pd.Series(l)
voc_df['AS처리유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,없음,리모콘사용법 미숙지,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5,없음,없음,없음,리모콘사용법 미숙지,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,없음,리모콘사용법 미숙지,False
3,고객불만,설치(이전)/AS,잦은 AS,2,위성,수신기 동작불가,수신기 전원 동작불가,공유기 사용 미숙지,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,없음,리모콘사용법 미숙지,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5,위성,없음,없음,리모콘사용법 미숙지,False
40484,고객불만,영업,상품/약정안내 미흡,3,위성,없음,없음,리모콘사용법 미숙지,False
40485,고객불만 이관,대한주부클럽연합회,영업,4,위성,없음,없음,리모콘사용법 미숙지,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7,위성,없음,없음,리모콘사용법 미숙지,False


In [85]:
# # 데이터를 읽고 순서를 섞는다
voc_model_df=voc_df.sample(frac=1).reset_index(drop=True)
voc_model_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,고객불만,설치(이전)/AS,(AS기사) 처리지연,1,위성,신호미약 메시지,전산 작업 오류,수신기 파손,False
1,고객불만,설치(이전)/AS,(AS기사) 불친절,3,위성,영상 안아놈,안테나 방향 틀어짐,리모콘 동작불가,False
2,고객불만 이관,과기정통부,영업,3,위성,없음,없음,리모콘사용법 미숙지,True
3,서비스 문의 및 신청,서비스 신청,SkyChoice,6,없음,없음,없음,리모콘사용법 미숙지,False
4,고객불만,영업,영업점불만,4,위성,없음,없음,리모콘사용법 미숙지,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만 이관,소비자연맹,해지,2,위성,없음,없음,리모콘사용법 미숙지,False
40484,고객불만,영업,영업점불만,5,위성,없음,없음,리모콘사용법 미숙지,True
40485,고객불만,영업,기타 단체,1,위성,없음,없음,리모콘사용법 미숙지,False
40486,고객불만 이관,본사 전화민원,설치(이전)/AS,2,위성,없음,없음,리모콘사용법 미숙지,False


In [86]:
voc_model_df.dtypes

상담대분류      object
상담중분류      object
상담소분류      object
급지         object
상품구분       object
AS신청유형     object
AS원인유형     object
AS처리유형     object
rscs_yn      bool
dtype: object

In [87]:
!pip install JPype1
!pip install konlpy

Looking in indexes: http://aicentro-nexus-svc:8081/hub-nexus/repository/pypi-group/simple
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://aicentro-nexus-svc:8081/hub-nexus/repository/pypi-group/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


In [88]:
from konlpy.tag import Kkma
kkma = Kkma()

In [89]:
cnslt_dic = print(kkma.nouns(u'고객불만 이관'))

['고객', '고객불만', '불만', '이관']


In [90]:
voc_df["상담"] = voc_df["상담대분류"] + voc_df["상담중분류"] + voc_df["상담소분류"]

In [91]:
voc_df['AS'] = voc_df["AS신청유형"] + voc_df['AS원인유형'] + ['AS처리유형']

In [92]:
voc_df['상담+AS'] = voc_df['상담'] + voc_df['AS']

In [93]:
val_columns = ['상담+AS','상담', 'AS', '급지', '상품구분', 'rscs_yn']
voc_df = pd.DataFrame(voc_df, columns=val_columns)

In [94]:
voc_df

,상담+AS,상담,AS,급지,상품구분,rscs_yn
0,서비스 문의 및 신청서비스 신청SkyChoice없음없음AS처리유형,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,4,없음,False
1,서비스 문의 및 신청서비스 신청SkyChoice없음없음AS처리유형,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,5,없음,False
2,서비스 문의 및 신청서비스 신청SkyChoice없음없음AS처리유형,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,3,없음,False
3,고객불만설치(이전)/AS잦은 AS수신기 동작불가수신기 전원 동작불가AS처리유형,고객불만설치(이전)/AS잦은 AS,수신기 동작불가수신기 전원 동작불가AS처리유형,2,위성,False
4,서비스 문의 및 신청서비스 신청SkyChoice없음없음AS처리유형,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,4,없음,False
...,...,...,...,...,...,...
40483,고객불만영업영업점불만(임의개통_이중가입)없음없음AS처리유형,고객불만영업영업점불만(임의개통_이중가입),없음없음AS처리유형,5,위성,False
40484,고객불만영업상품/약정안내 미흡없음없음AS처리유형,고객불만영업상품/약정안내 미흡,없음없음AS처리유형,3,위성,False
40485,고객불만 이관대한주부클럽연합회영업없음없음AS처리유형,고객불만 이관대한주부클럽연합회영업,없음없음AS처리유형,4,위성,False
40486,고객불만영업영업점불만(임의개통_일방적가입)없음없음AS처리유형,고객불만영업영업점불만(임의개통_일방적가입),없음없음AS처리유형,7,위성,False


In [95]:
nlp_dic = []
#ll = [kkma.nouns(i) for i in voc_df['상담'].head(1000)]
def add_dic(s):
    global nlp_dic
    nlp_dic=list(set(nlp_dic+list(s)))
#     print(nlp_dic)
for i in voc_df['상담'].head(10):
    ii=set(kkma.nouns(i))
    add_dic(ii)
    
for i in voc_df['AS'].head(10):
    ii=set(kkma.nouns(i))
    add_dic(ii)
    
print('---')    
print(nlp_dic)

---
['신청서비스', '동작불가수신기', '서비스', '고객', '유형', '수신기', '동작불가', '전원', '설치', '이전', '처리', '불만', '문의', '동작', '고객불만설치', '처리유형', '신청', '불가']


In [96]:
nlp_dic = []
nlp_dic = ['클레임', '칭찬기타동', '환경', '동축콘넥터', '환불', '발생수신기', '속도저하', '휴일', '중복발생수신기', '침해', '메시지일시적', '환불리모콘', '이관내방고객설치', '오연결', '회사', '단체방송품질', '불친절기타', '이행', '장비', '이관소비자보호원영업', '동작불가잭', '연결', '파전환방법', '공', '이관기타', '전화민원방송품질', '불', '기타동', '불량자상파전환방법', '기정통부해지', '유통망관련', '이관소비자신문기타', '스마트', '미흡영상', '미이행기타', '영업점불만', '구내', '이관소비자보호원해지', '연맹', '지연영상', '시청불가', '주부클럽연합회', '절', '단체영상', '청구', '신호미약', '환불신호미약', '전화민원영업영상', '모임명의도용', '통신', '녹화', '미동작리모콘사용법', '메시지동작불가', '연결요청', '미이행', '수신', '제도', '민원', '위험지역', '미이행영상', '메시지리모콘', '임의개통', '모임방송', '연결요청리모콘사용법', '메시지불량', '점불만', '영상불량', '단체이용불편', '비고장', '정책', '불만기타', '신청패키지', '단체방송품질수신기', '해지', '미지금', '기정', '임의', '중복발생신호미약', '점불만재해지역', '점불만수신기', '미동작리모콘', '시청불가음성', '단체재해지역', '전화민원상담음성', '자연', '등', '모름회선', '영상불량기타리모콘', '정오', '분실스마트카드', '후', '보호원이용', '타스', '커넥터', '상담', '미흡기타', '방송품질수신기', '분실동축콘넥터', '가스', '구내단자함', '콘텐츠리모콘', '주의', '방문약속', '산', '지연', '이관소비자보호원청구', '고객부주의', '모름케이블연결방법', '메시지안테나', '메시지수신기', '이관대', '이관소비자신문영업', '영상', '불친절', '미동작동', '일방적가입', '단체기타동', '서비스', '모임해지영상', '클레임처리지연', '전원', '미', '패키지', '미이행수신기', '미흡케이블', '영업점', '특이', '명의', '내방', '설치점불만수신', '동갖불가', '미흡기타동축', '기정통부이용', '고객불만설치', '불친절기타리모콘사용법', '지연수신기', '미동작전', '놈파손', '개인정보', '미사용', '회복', '추가', '점검', '불만수신기', '사용법', '모임해지', '녹화방법', '가입해지', '미이행케이블', '해지음성', '전화민원상담리모콘', '미흡스마트카드', '미동작수신기', '카드', '전화민원해지수신', '정상', '전화민원해지수신기', '단체영업수신기', '타동', '네트워크', '이전', '미이행재해지역', '클럽', '가입', '오류', '불친절수신설비', '이용불편영상', '모름리모콘', '해지수신기', '백업', '불량동작', '공법', '작업', '취소', '콘', '사례', '분실파손', '주부클럽연합회기타', '모임영업영상', '작전', '단체이용불편영상', '단체기타수신기', '미흡기타전산', '쌍방향', '전화민원해지영상', '방송', '메시지파손', '약정안내', '분실수신설비', '메시지동', '메시지기타', '미동작스마트카드', '유통망', '미이행기타스마트', '도서', '놈지상파', '점불만스마트카드', '잡음증폭기', '품질', '놈리모콘', '취약', '단체해지기타', '놈동축콘넥터', '불량녹화방법', '모임청구', '불만', '메시지회선', '방문', '축콘넥터', '처리누락', '기타전산', '저하', '처리지연영상', '콘넥터', '불만영상', '환불영상', '기정통부해지영상', '피해', '확인', '분실리모콘', '처리지연기타전산', '동작불가스마트카드', '전화', '오', '단체기타', '전화민원영업기타', '손괴신호미', '고객요청', '전화민원콘텐츠', '미흡기타일시적', '방수처리', '모름', '일방적', '불만리모콘', '분식', '납', '불량동', '기타일시적', '메시지전원', '증폭기', '점불만음성', '기정통부해지기타', '모름파손', '기타', '불량설치관련', '음성불량음성', '시청불가영상', '신호', '놈', '처리지연케이블', '메시지취약지역', '모름안테나', '기정통부해지음성', '환불재해지역', '짐', '메시지도시가스배관', '처리', '콘텐츠', '불량리모콘', '비고', '미약', '소비자연맹방송', '불가', '지금', '전환', '점불만인터넷', '고객사유', '사용', '환불스마트카드', '관련', '메시지자상파전환', '놈케이블연결방법', '분실케이블연결방법', '불만기타안테나', '주부클럽연합회콘텐츠', '단체방송품질기타', '단체음성', '유발', '모름동', '요청', '불가리모콘사용법', '방송품질영상', '전화민원해지신호', '분실전산', '세대', '영업업셀링', '공사', '단체상담', '불량리모콘사용법', '품질리모콘', '전화민원영업', '제품', '전화민원영업케이블', '분실', '메시지케이블연결방법', '단체리모콘', '복구', '미이행기타수신기', '미흡수신설비', '전화민원해지', '가격', '영업수신설비', '인터넷', '분배', '시민', '품질수신장애', '도서지역', '허브', '단체스마트카드', '셀', '처리지연스마트카드', '설치기사', '동작불가리모콘', '이용불편', '설치점불만케이블', '미이행수신설비', '전화민원상담케이블', '불만음성', '영업', '단선', '불량자', '이관소비자연맹청구', '지연기타수신기', '분실일시적', '전화민원기타', '영업수신기', '품질음성', '영업영상', '처리점', '사용불가', '전화민원청구', '설치지연', '동작불가파손', '불친절음성', '모임해지수신기', '모임가격정책', '환불음성', '불편리', '단체해지수신기', '점불만기타동', '케이블', '미흡기타안테나', '해지신호미약', '동축', '불량안테나', '이관내방고객가격', '방송품질', '안됨', '잭', '포트', '불가파손', '단체영업기타', '상파', '불친절기타스마트카드', '미동작일시적', '미흡기타케이블연결', '리모콘', '정상확인', '영업리모콘', '점불만신호미약', '단체설치', '놈수신기', '접', '소비자연맹', '결합', '포트불량', '동갖', '출동지연', '제조사', '단체수신설비', '타임머신', '동작불가신호', '영상불량케이블', '기정통부설치', '불편영상', '모뎀', '위원회', '손괴', '처리지연신호미약', '모', '공법불량', '분실녹화방법', '주부', '준수', '모름일시적', '동작불가수신기', '점검리모콘사용법', '축', '동작불가전산', '발생리모콘사용법', '수신설비', '리모콘리모콘', '점불만실버리모콘', '문의', '주부클럽연합회방송', '기정통부상담리모콘', '불량동축콘넥터', '마트', '플레이어', '미흡기타리모콘사용법', '환불기타일시적', '수신기', '불가녹화방법', '영업점불만기타', '전화민원해지기타', '설치', '지상파', '이행기', '단체청구', '호', '문의고객센터', '이관소비자연맹해지', '동작', '공유기', '신청기업', '분실수신기', '불친절기타안테나', '약속', '주부클럽연합회설치', '설치관련', '유니트', '기정통부방송', '미숙지', '기타리모콘사용법', '불량해상도', '메시지공유기', '설치점불만리모콘', '놈스마트카드', '고장', '배관', '미이행음성', '기타메시지기타', '설치점불만영상', '미이행신호미', '불만신호미', '문의기사칭찬', '분실리모콘사용법', '손괴재해지역', '기정통부이용불편', '방송품질기타', '볼트', '미이행기타안테나', '메시지가스배관', '접불', '이관내방고객영업', '단체영업', '일시적', '회선', '공급', '미동작자상파전환', '기정통부', '신청서비스', '처리지연기타', '입선', '환불수신기', '메시지리모콘사용법', '채널', '방법', '개통', '놈동', '점', '영업점불만케이블', '기타설치관련', '기타파손', '동작불가설치관련', '이용', '보호원설치', '이중가입', '단체', '불만기타수신기', '미흡기타동', '모임상담', '네트워크장비장애', '메시지동작', '해상도', '불친절인터넷', '정보', '장', '미동작', '동작불가동축', '칭찬기타', '지연신호미약', '단체이용불편리모', '출동', '전화민원가격정책', '놈일시적', '이관소비자연맹설치', '점불만기타일시적', '지연기타리모콘사용법', '단체기타공유기', '영업케이블', '단체영업영상', '지연음성', '지역', '리모콘사용법', '주부클럽연합회해지', '불량파손', '불량불량', '불량', '연결확인', '사은품', '동작불가', '호전환해지', '전화민원상담', '동작자', '기술상담', '불만기타리모콘사용법', '메시지접속불량', '커넥터불량', '시설', '기타케이블연결방법', '고장불량', '유형', '동작불가안테나', '모름리모콘사용법', '불친절재해지역', '주부클럽연합회청구', '재해지역', '손괴영상', '해지기타', '불만케이블', '기정통부가격정책', '설치점불만음성', '앙카', '불친절신호미', '작동', '소비자', '발생안테나', '모임해지신호미약', '기', '소비자보호원', '미작동수신기', '단체해지음성', '주부클럽연합회가격', '불친절영상', '미흡인터넷', '불가안테나', '미이행기타리모콘', '품질기타', '모름동축콘넥터', '약속미이행', '놈설치관련', '설치점불만', '기업', '약정', '단체인터넷', '직원', '단자함', '민원영업', '단체영업리모콘', '미동작안테나', '고객취소', '재해', '칭찬리모콘', '미흡수신기', '분실안테나', '침해신호미', '놈동작', '메시지동축콘넥터', '처리지연음성', '민원방송', '이관방송통신위원회', '단체기타전산', '동', '동작동', '약속미준수', '신문', '입광', '모임이용', '누락', '불친절수신기', '설치점불만기타', '단체해지신호미', '보호원', '불량기타', '모임', '신청쌍방향', '기타수신기', '중복발생영상', '센터', '시청', '점불만영상', '이관소비자신문청구', '후처리', '업그레이드안됨수신기', '기정통부영업', '소비자연맹해지', '환불케이블', '처리지연', '처리지연리모콘', '환경설정오', '설', '이관내방고객청구', '불만기', '이관시민', '불량일시적', '이중', '시설노후', '불편', '불친절리모콘', '놈자상파전환', '실내선', '모임개인정보', '동작불가일시적', '대', '미이행인터넷', '처리유형', '방향', '통부', '이관본사', '영업상품', '이용불편기타', '케이블연결방법', '시청불가기타', '기정통부청구', '점불만기타수신기', '절연불량', '이관소비자신문해지', '이관소비자연맹콘텐츠', '전화민원설치', '메시지녹화방법', '분실동', '상담신호미약', '칭찬수신기', '발생', '도시', '전화민원콘텐츠영상', '신호미', '중복발생기타', '업그레이드', '대필', '점불만기타', '동작불가녹화방법', '문의기사', '처리지연기타수신기', '세대단자함', '콘텐츠수신기', '연결안됨', '연합회', '영업사은품', '민원상담', '모름설치관련', '품질수신기', '모임방송품질', '단체콘텐츠', '실버', '불만기타파손', '잡음', '단체가격정책', '처리지연인터넷', '분리', '이관소비자보호원기타', '환불기타설치관련', '단체방송', '케이', '정통부', '미흡리모콘', '미동작케이블연결', '위험', '안테나', '칭찬', '지연리모콘', '불만기타동', '접수', '점불만기타안테나', '정비', '주부클럽', '속도', '신호미약수신기', '체계', '점불만리모콘', '미이행기타전산', '연', '불편수신설비', '해지기타리모콘사용법', '부', '명의도용', '놈녹화방법', '전화민원상담영상', '환불수신설비', '미흡', '불친절케이블', '비용', '처리지연수신설비', '불복구', '안내', '해지영상', '점불만기타파손', '모름전산', '불친절스마트카드', '숙지', '기타안테나', '처리지연기타리모콘', '불친절기타파손', '개인', '불편수신기', '단체방송품질영상', '신청', '파', '중복', '동작불가동', '접속불량', '파손', '영업가', '단체클레임처리지연', '변경', '발생일시적', '해지기타수신기', '불편기타', '오처리', '불만기타전산', '점불만기타리모콘', '설치점', '이관소비자연맹가격', '놈신호', '미이행모뎀', '전화민원이용불편', '놈안테나', '단체영업인터넷', '채널설정방법', '연결방법', '기사', '미동작파손', '실버리모콘리모콘', '단체케이블', '환불기타수신기', '처리지연수신기', '전원공급', '사용미숙', '신호미약기타', '미동', '요금', '방수', '미숙', '영업기타', '음성', '불만수신설비', '메시지설치관련', '메시지스마트카드', '처리지연기타동', '메시지신호', '주부클럽연합회상담', '놈리모콘사용법', '놈해상도', '백업고객', '기술', '처리지연기타안테나', '설비', '단체수신기', '전화민원해지음성', '불량접속불량', '이관소비자연맹영업', '모름수신기', '환불기타', '불가전산', '놈전산', '미흡음성', '장애', '특이사례', '모임영업', '모임해지기타', '자상', '스마트카드', '영업기타수신기', '단체방송품질리모콘', '미이행리모콘', '칭찬케이블', '앙카볼트', '이관소비자보호원가격', '불량케이블연결방법', '불량동작불가', '불량전산', '단체신호미', '점불만수신설비', '이관소비자연맹기타', '유발고객재산', '불량수신기', '미흡신호미', '단체이용', '미이행스마트카드', '불가수신기', '전산', '전', '업', '폐문', '본사', '영업가전결합', '비용불만', '미이행네트워크', '미이행기타파손', '재산', '이관', '설치점불만수신기', '불편음성', '입', '소비자연맹이용', '고객', '메시지전산', '노후', '제품불량', '단체해지', '지연기타', '불편리모콘', '영상불량영상', '주부클럽연합회영업', '기정통부기타', '연합회해지수신기', '고객불만', '불편기타수신기', '기정통부상담', '영업음성', '유기', '환불기타리모콘사용법', '가입자', '상품', '모름스마트카드', '주부클럽연합회이용', '품질영상', '민원이용', '자연회복', '가격정책', '불량신호', '사유', '링', '중복발생', '연결요청수신기', '단체기타리모콘사용법', '미흡기타수신기', '설정', '미흡기타파손', '점불만모뎀', '모임설치', '점불만기타동축', '단체해지영상', '점불만케이블', '미흡재해지역', '도용', '광', '모임해지기타수신기', '접속', '메시지']

In [97]:
import pickle

In [37]:
 
## Save pickle
with open("nlp_dic_2.pickle","wb") as fw:
    pickle.dump(nlp_dic, fw)

In [38]:
 
## Load pickle
with open("nlp_dic.pickle","rb") as fr:
    data = pickle.load(fr)
nlp_dic = data    
print(nlp_dic)

['클레임', '칭찬기타동', '환경', '동축콘넥터', '환불', '발생수신기', '속도저하', '휴일', '중복발생수신기', '침해', '메시지일시적', '환불리모콘', '이관내방고객설치', '오연결', '회사', '단체방송품질', '불친절기타', '이행', '장비', '이관소비자보호원영업', '동작불가잭', '연결', '파전환방법', '공', '이관기타', '전화민원방송품질', '불', '기타동', '불량자상파전환방법', '기정통부해지', '유통망관련', '이관소비자신문기타', '스마트', '미흡영상', '미이행기타', '영업점불만', '구내', '이관소비자보호원해지', '연맹', '지연영상', '시청불가', '주부클럽연합회', '절', '단체영상', '청구', '신호미약', '환불신호미약', '전화민원영업영상', '모임명의도용', '통신', '녹화', '미동작리모콘사용법', '메시지동작불가', '연결요청', '미이행', '수신', '제도', '민원', '위험지역', '미이행영상', '메시지리모콘', '임의개통', '모임방송', '연결요청리모콘사용법', '메시지불량', '점불만', '영상불량', '단체이용불편', '비고장', '정책', '불만기타', '신청패키지', '단체방송품질수신기', '해지', '미지금', '기정', '임의', '중복발생신호미약', '점불만재해지역', '점불만수신기', '미동작리모콘', '시청불가음성', '단체재해지역', '전화민원상담음성', '자연', '등', '모름회선', '영상불량기타리모콘', '정오', '분실스마트카드', '후', '보호원이용', '타스', '커넥터', '상담', '미흡기타', '방송품질수신기', '분실동축콘넥터', '가스', '구내단자함', '콘텐츠리모콘', '주의', '방문약속', '산', '지연', '이관소비자보호원청구', '고객부주의', '모름케이블연결방법', '메시지안테나', '메시지수신기', '이관대', '이관소비자신문영업', '영상', '불친절', '미동작동', '일방적가입', '단체기타동', '서

######## tf-idf 적용 #######

In [98]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
#transformer = TfidfTransformer(smooth_idf=False)  #IDF값의 분모와 분자에 1을 더하는 인수
#transformer = TfidfTransformer(use_idf=True)
#transformer = TfidfTransformer(sublinear_tf=True)
transformer = TfidfTransformer(sublinear_tf=True)

dir(transformer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_n_features',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_data',
 'fit',
 'fit_transform',
 'get_params',
 'idf_',
 'norm',
 'set_params',
 'smooth_idf',
 'sublinear_tf',
 'transform',
 'use_idf']

In [ ]:
vector = CountVectorizer()
vector.fit_transform(nlp_dic)
nlp_dic

In [108]:
a = np.zeros(len(nlp_dic)) 
a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [118]:
nlp_dic_enc={k:i for i,k in enumerate (nlp_dic)}
nlp_dic_enc

{'클레임': 0,
 '칭찬기타동': 1,
 '환경': 2,
 '동축콘넥터': 3,
 '환불': 4,
 '발생수신기': 5,
 '속도저하': 6,
 '휴일': 7,
 '중복발생수신기': 8,
 '침해': 9,
 '메시지일시적': 10,
 '환불리모콘': 11,
 '이관내방고객설치': 12,
 '오연결': 13,
 '회사': 14,
 '단체방송품질': 15,
 '불친절기타': 16,
 '이행': 17,
 '장비': 18,
 '이관소비자보호원영업': 19,
 '동작불가잭': 20,
 '연결': 21,
 '파전환방법': 22,
 '공': 23,
 '이관기타': 24,
 '전화민원방송품질': 25,
 '불': 26,
 '기타동': 27,
 '불량자상파전환방법': 28,
 '기정통부해지': 29,
 '유통망관련': 30,
 '이관소비자신문기타': 31,
 '스마트': 32,
 '미흡영상': 33,
 '미이행기타': 34,
 '영업점불만': 35,
 '구내': 36,
 '이관소비자보호원해지': 37,
 '연맹': 38,
 '지연영상': 39,
 '시청불가': 40,
 '주부클럽연합회': 41,
 '절': 42,
 '단체영상': 43,
 '청구': 44,
 '신호미약': 45,
 '환불신호미약': 46,
 '전화민원영업영상': 47,
 '모임명의도용': 48,
 '통신': 49,
 '녹화': 50,
 '미동작리모콘사용법': 51,
 '메시지동작불가': 52,
 '연결요청': 53,
 '미이행': 54,
 '수신': 55,
 '제도': 56,
 '민원': 57,
 '위험지역': 58,
 '미이행영상': 59,
 '메시지리모콘': 60,
 '임의개통': 61,
 '모임방송': 62,
 '연결요청리모콘사용법': 63,
 '메시지불량': 64,
 '점불만': 65,
 '영상불량': 66,
 '단체이용불편': 67,
 '비고장': 68,
 '정책': 69,
 '불만기타': 70,
 '신청패키지': 71,
 '단체방송품질수신기': 72,
 '해지': 73,
 '미지금': 74,
 '

In [128]:
r=a.copy()
r[1]=r[1]+1
r
voc_df['상담'][1:2]

1    서비스 문의 및 신청서비스 신청SkyChoice
Name: 상담, dtype: object

In [132]:
import pickle

In [ ]:
c1=[]
for i in voc_df['상담'] :
#i="서비스 문의 및 신청서비스 신청SkyChoice"
    ii=kkma.nouns(i) 
    r=a.copy()
    for j in ii:
        try:
            n=nlp_dic_enc[j]
        except KeyError:
            pass
        r[n]=r[n]+1
    c1.append(r)

## Save pickle
with open("nlp_cnslt.pickle","wb") as fw:
    pickle.dump(c1, fw)

In [ ]:
c2=[]
for i in voc_df['AS'] :
#i="서비스 문의 및 신청서비스 신청SkyChoice"
    ii=kkma.nouns(i) 
    r=a.copy()
    for j in ii:
        try:
            n=nlp_dic_enc[j]
        except KeyError:
            pass
        r[n]=r[n]+1
    c2.append(r)

## Save pickle
with open("nlp_as.pickle","wb") as fw:
    pickle.dump(c2, fw)

In [ ]:
c2=[]
for i in voc_df['AS'] :
#i="서비스 문의 및 신청서비스 신청SkyChoice"
    ii=kkma.nouns(i) 
    r=a.copy()
    for j in ii:
        try:
            n=nlp_dic_enc[j]
        except KeyError:
            pass
        r[n]=r[n]+1
    c1.append(r)


In [106]:
cnslt_array = vector.transform(voc_df['상담']).toarray()

cnslt_array.shape

(40488, 809)

In [101]:
voc_df['상담']

0        서비스 문의 및 신청서비스 신청SkyChoice
1        서비스 문의 및 신청서비스 신청SkyChoice
2        서비스 문의 및 신청서비스 신청SkyChoice
3                고객불만설치(이전)/AS잦은 AS
4        서비스 문의 및 신청서비스 신청SkyChoice
                    ...            
40483        고객불만영업영업점불만(임의개통_이중가입)
40484              고객불만영업상품/약정안내 미흡
40485            고객불만 이관대한주부클럽연합회영업
40486       고객불만영업영업점불만(임의개통_일방적가입)
40487        고객불만영업영업점불만(임의개통_이중가입)
Name: 상담, Length: 40488, dtype: object

In [77]:
tfidf = transformer.fit_transform(cnslt_array)
cnslt_array=tfidf.toarray()
cnslt_array
cnslt_array.shape

(40488, 809)

In [78]:
cnslt_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [79]:
d=vector.vocabulary_

In [80]:
# cnslt_df = pd.DataFrame(cnslt_array,columns=d.keys())

cnslt_df = pd.DataFrame(cnslt_array)

In [118]:
cnslt_df.head(333).tail(100)

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
as_array = vector.transform(voc_df['AS']).toarray()

as_array.shape

(40488, 809)

In [83]:
tfidf = transformer.fit_transform(as_array)
as_array=tfidf.toarray()
as_array.shape

(40488, 809)

In [84]:
as_df = pd.DataFrame(as_array) #,columns=d.keys())

as_df

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
val_nlp_columns = ['급지', '상품구분', 'rscs_yn']
voc_nlp_df = pd.DataFrame(voc_df, columns=val_nlp_columns)
voc_nlp_df.columns=['gupji','prdttype','rscs_yn']

In [52]:
# 속도 개선을 위해 파이프라인을 사용하도록 개선
# 참고 : https://stackoverflow.com/questions/28160335/plot-a-document-tfidf-2d-graph
pipeline = Pipeline([
    ('vect', vectorizer),
]) 

NameError: name 'vectorizer' is not defined

In [85]:
as_df

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
as_df.columns=[100000+i for i in as_df.columns]

In [87]:
as_df

,100000,100001,100002,100003,100004,100005,100006,100007,100008,100009,...,100799,100800,100801,100802,100803,100804,100805,100806,100807,100808
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
cnslt_df

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
voc_nlp_df

,gupji,rscs_yn
0,4,False
1,5,False
2,3,False
3,2,False
4,4,False
...,...,...
40483,5,False
40484,3,False
40485,4,False
40486,7,False


In [57]:
del voc_nlp_df['prdttype']

In [90]:
#pd.concat(voc_nlp_df,as_df)
voc_final_df = pd.concat([as_df, cnslt_df,voc_nlp_df], axis=1)
voc_final_df

,100000,100001,100002,100003,100004,100005,100006,100007,100008,100009,...,801,802,803,804,805,806,807,808,gupji,rscs_yn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,False
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,False
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,False
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,False
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,False
40484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,False
40485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,False
40486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,False


In [91]:
voc_train=voc_final_df.iloc[0:20000,:]
voc_test=voc_final_df.iloc[20000:40488,:]

In [92]:
x_train= voc_train.iloc[:,0:1619].values
x_test= voc_test.iloc[:,0:1619].values
y_train= voc_train.iloc[:,1619:1620]
y_test= voc_test.iloc[:,1619:1620]

In [93]:
y_train

,rscs_yn
0,False
1,False
2,False
3,False
4,False
...,...
19995,True
19996,False
19997,False
19998,False


In [94]:
# encoder={k:v for v,k in enumerate(y_train.drop_duplicates())}
# encoder
sets=voc_final_df.iloc[:,1619:1620].drop_duplicates()["rscs_yn"].tolist()

In [95]:
encoder={k:v for v,k in enumerate(sets)}

In [96]:
y_train=[ encoder[i] for i in y_train["rscs_yn"].tolist() ]

In [97]:
batch_size = 50
num_classes = 3
epochs = 50

In [98]:
y_train = keras.utils.to_categorical(y_train, num_classes)

y_test=[ encoder[i] for i in y_test["rscs_yn"].tolist() ]
y_test = keras.utils.to_categorical(y_test, num_classes)

In [99]:
y_test

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [100]:
# 데이터 shape 을 확인해 봅니다.
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(20000, 1619) (20000, 3) (20488, 1619) (20488, 3)


In [101]:
model = Sequential()
model.add(Dense(18, activation='relu', input_shape=(1619,)))
model.add(Dropout(0.4))
model.add(Dense(20, activation='tanh',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(5, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
#'binary_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 18)                29160     
_________________________________________________________________
dropout_3 (Dropout)          (None, 18)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                380       
_________________________________________________________________
dropout_4 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 105       
_________________________________________________________________
dropout_5 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                

In [102]:
import  tensorflow.keras.optimizers
dir( tensorflow.keras.optimizers)

['Adadelta',
 'Adagrad',
 'Adam',
 'Adamax',
 'Ftrl',
 'Nadam',
 'Optimizer',
 'RMSprop',
 'SGD',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_deprecation_wrapper',
 '_sys',
 'deserialize',
 'get',
 'schedules',
 'serialize']

In [103]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 20000 samples, validate on 20488 samples
Epoch 1/50
20000/20000 [==============================] - 26s 1ms/sample - loss: 0.3805 - acc: 0.9024 - val_loss: 0.3128 - val_acc: 0.8882
Epoch 2/50
20000/20000 [==============================] - 27s 1ms/sample - loss: 0.2901 - acc: 0.9038 - val_loss: 0.3060 - val_acc: 0.8882
Epoch 3/50
20000/20000 [==============================] - 28s 1ms/sample - loss: 0.2831 - acc: 0.9042 - val_loss: 0.3040 - val_acc: 0.8882
Epoch 4/50
14100/20000 [====================>.........] - ETA: 4s - loss: 0.2811 - acc: 0.9042

KeyboardInterrupt: 

In [300]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.29724920480244793
Test accuracy: 0.88816214
